In [ ]:
%pip install termcolor  langchain_experimental tiktoken sentence-transformers

In [ ]:
#!git clone https://github.com/facebookresearch/faiss.git

In [ ]:
!sudo apt-get -y install libblas-dev liblapack-dev swig


In [ ]:
%pip install faiss-gpu

In [ ]:
#!rm -rf build/
#!mkdir build

In [ ]:
#%cd /notebooks/faiss

In [ ]:
#%cd /notebooks/faiss
#!cmake  -B build .

In [ ]:
#!make -C build -j faiss

In [ ]:
#!make -C build -j swigfaiss
##%cd build/faiss/python
#%run setup.py install
#%cd /notebooks

In [ ]:
%reload_ext dotenv
%dotenv /notebooks/chainlang/my.env

In [ ]:
import logging

logging.basicConfig(level=logging.ERROR)

In [ ]:
from datetime import datetime, timedelta
from typing import List
from termcolor import colored


from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.vectorstores import FAISS

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

LLM=LlamaCpp(
    model_path="/notebooks/zephyr-7b-alpha.Q8_0.gguf",
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=False,
    n_ctx=12000,
    max_tokens=4500,
    n_gpu_layers=65
)


In [ ]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

LLM_MEMORY=LlamaCpp(
    model_path="/notebooks/zephyr-7b-alpha.Q8_0.gguf",
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=False,
    callback_manager=callback_manager,
    n_ctx=2800,
    max_tokens=4500,
    n_gpu_layers=34,
    temperature=0
)

In [ ]:
context='''
Here the context for the characters personalities and story:
'''

In [ ]:
from langchain.memory import ConversationBufferMemory


from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.chains import LLMChain
from langchain.chains import ConversationChain  
from langchain.memory import ConversationSummaryMemory, CombinedMemory,VectorStoreRetrieverMemory

template=f'''
Welcome to the Interactive Chat System! Here, you can engage in a guided story with our AI friend.
The goal of the system is to help writing an story. 
The user input dialogue in a format or request descriptions of scenes and actions.
The system will also respond or react as  character's specified by [AI]CHARACTER NAME[/AI] based on described human actions. Let's begin!
[EXAMPLE]
    Human:
    [DIALOG] A to B: Hey, how are you? [/DIALOG]
    [AI]B[/AI]    
    [DIALOG] A to B: Hihihi *giggle* Fine and you ? [/DIALOG]
    Human:
    [ACTION] Look at A [/ACTION]
    [DIALOG]: A to B: Wanna go grab something to eat ? [/DIALOG]
    [AI] B  [/AI]     

    [ACTION] Look around while responding[/ACTION]
    [DIALOG] A to B: yep let's go [/DIALOG]
    Human:
    [DESCRIPTIONS] Describe the action[/DESCRIPTIONS]
    [AI] B  [/AI]      
  
    [DESCRIPTIONS] On a calm and uneventful Saturday morning, A found himself standing before his neighbor's plain, white door. The street was quiet, the usual bustle of the neighborhood had not yet begun. The sky overhead was a clear blue, with the sun casting a warm, gentle light on the houses lining the street. The birds chirped cheerily, oblivious to the human dramas unfolding below them.

A was about to knock on the door when a sudden shout broke the tranquility of the morning [/DESCRIPTIONS]   
[/EXAMPLE]
'''
memory_template = """
[CONTEXT]
{context}
[/CONTEXT]

"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    SystemMessagePromptTemplate.from_template(memory_template),
    AIMessagePromptTemplate.from_template("Alright, let's get started..."),
    MessagesPlaceholder(variable_name="chat_summary"),
    MessagesPlaceholder(variable_name="chat_history"),
    human_message_prompt])


In [ ]:
from langchain.memory import ConversationBufferMemory


from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.chains import LLMChain
from langchain.chains import ConversationChain  
from langchain.memory import ConversationSummaryMemory, CombinedMemory,VectorStoreRetrieverMemory

template=f'''
 
'''
memory_template = """
[CONTEXT]
{context}
[/CONTEXT]

"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    SystemMessagePromptTemplate.from_template(memory_template),
    AIMessagePromptTemplate.from_template("Alright, let's get started..."),
    MessagesPlaceholder(variable_name="chat_summary"),
    MessagesPlaceholder(variable_name="chat_history"),
    human_message_prompt])


In [ ]:
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)
import math
import faiss
from langchain.embeddings import GPT4AllEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.memory import ConversationSummaryMemory, VectorStoreRetrieverMemory

def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return score


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Initialize the vectorstore as empty
    embedding =  HuggingFaceEmbeddings(
            model_name="intfloat/multilingual-e5-large"
    )
    embedding_size = 1024
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(embedding, index, InMemoryDocstore({}), {})
    retriever = vectorstore.as_retriever(
        search_type="mmr", # Also test "similarity"
        search_kwargs={"k": 8},
    )
    return VectorStoreRetrieverMemory(retriever=retriever,input_key="text",memory_key="chat_memory",return_messages=False,output_key="chat_history")

In [ ]:
chat_memory = create_new_memory_retriever()
current_chat_memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True,input_key="text",output_key="text",max_history=4)
summary_memory = ConversationSummaryMemory(llm=LLM_MEMORY,memory_key="chat_summary",return_messages=True,input_key="text")
memory = CombinedMemory(memories=[current_chat_memory, summary_memory])

In [ ]:

conversation = LLMChain(
    llm=LLM,
    prompt=chat_prompt,
    verbose=True,
    memory=memory
)


In [ ]:

conversation({"text": '''
[INSTRUCTION]
[/INSTRUCTION]
[AI][/AI]
''',"context":context})

In [ ]:
current_chat_memory.chat_memory.messages.pop(0)

In [ ]:
current_chat_memory.chat_memory.messages.pop(len(current_chat_memory.chat_memory.messages)-1)

In [ ]:
summary_memory.clear()

In [ ]:
current_chat_memory.clear()
summary_memory.clear()

In [ ]:
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import os
def reset_all_memory():
    current_chat_memory.clear()
    summary_memory.clear()
    summary_memory.clear()
def reset_conversation():
    current_chat_memory.clear()
def forget_last_message():
    current_chat_memory.chat_memory.messages.pop(0)
def predict(message, history,context, actions, descriptions, ai):
    # Initialize an empty list to collect the formatted sections
    formatted_sections = []

    # Check each variable and append its formatted section to formatted_sections
    if message:
        formatted_sections.append(f'[DIALOG]\n{message}\n[/DIALOG]')
    if actions:
        formatted_sections.append(f'[ACTION]\n{actions}\n[/ACTION]')
    if descriptions:
        formatted_sections.append(f'[DESCRIPTION]\n{descriptions}\n[/DESCRIPTION')
    if ai:
        formatted_sections.append(f'[AI]\n{ai}\n[/AI]')

    # Join the formatted sections with newline characters to create formatted_input
    formatted_input = '\n'.join(formatted_sections)

    # The rest of your code remains the same
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    # Getting the response from langchain
    gpt_response = conversation({"text": formatted_input,"context":context})
    for message in gpt_response["chat_history"]:
        history_langchain_format.append(message)
    print(gpt_response)
    return gpt_response["text"]

# Defining additional input fields
additional_inputs = [
    gr.Textbox(default="Enter actions here", label="Actions",lines=2),
    gr.Textbox(default="Enter descriptions here", label="Descriptions",lines=2),
    gr.Textbox(default="Enter AI input here", label="AI",lines=2),
    gr.Textbox(default="Context", label="Context",lines=5,value=context),
    gr.Button(label="Forget last message", on_click=forget_last_message),
    gr.Button(label="Reset Conversation", on_click=reset_conversation),
    gr.Button(label="Reset all memory", on_click=reset_all_memory),
    
]

# Creating the ChatInterface with additional input fields
chat_interface = gr.ChatInterface(
    fn=predict, 
    additional_inputs=additional_inputs
)

chat_interface.launch(share=False)

OLD

In [ ]:
tommies_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=12,  # we will give this a relatively low number to show how reflection works
)

tommie = GenerativeAgent(
    name="Minoru",
    age=20,
    traits="anxious, likes design, talkative",  # You can add more persistent traits here
    status="looking for job",  # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=tommies_memory,
)

In [ ]:
print(tommie.get_summary())

In [ ]:
print(tommie.get_summary())

In [ ]:
from langchain.vectorstores.utils import filter_complex_metadata
# We can add memories directly to the memory object
tommie_observations = [

]
for observation in tommie_observations:
    tommie.memory.add_memory(observation)

In [ ]:
observations = [

]

In [ ]:
# Let's send Tommie on their way. We'll check in on their summary every few observations to watch it evolve
for i, observation in enumerate(observations):
    _, reaction = tommie.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % 20) == 0:
        print("*" * 40)
        print(
            colored(
                f"After {i+1} observations, Minoro's summary is:\n{tommie.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)